<a href="https://colab.research.google.com/github/gapf1/examples/blob/main/Env_Trade_Historic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tf_agents.environments import py_environment

# Inpirada no texto em  https://www.tensorflow.org/agents/tutorials/2_environments_tutorial#creating_your_own_python_environment 

class Env_Trade_Historic(py_environment.PyEnvironment):
    
  def __init__(self):
    
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=0, maximum=1, name='action')

    self.look_back_window = 32
    
    self._observation_shape = (3072,)
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=self._observation_shape, dtype=np.float32, name='observation')
    
    self.open, self.high, self.low, self.close, self.change  = load_data(bars = 8760) # 50000
    self._state = np.zeros(self._observation_shape, dtype=np.float32) 
    self._episode_ended = False
    self.bar = 0

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = np.zeros(self._observation_shape, dtype=np.float32)        #0.0,0.0,0.0,0.0]
    self._episode_ended =    False
    return ts.restart(np.array(self._state, dtype=np.float32))


  def _step(self, action):
    
    reward = 0

    if self._episode_ended:
    # The last action ended the episode. Ignore the current action and start
    # a new episode.
        return self.reset() 

    self.bar += 1
    
    if self.bar >= len(self.open)- self.look_back_window:
        self.bar=0

    self._state =   createImage(  self.open[ self.bar:self.bar+self.look_back_window],
                                  self.high[ self.bar:self.bar+self.look_back_window],
                                  self.low[  self.bar:self.bar+self.look_back_window],
                                  self.close[self.bar:self.bar+self.look_back_window],
                                )
    X  =  self.change[self.bar:self.bar+self.look_back_window]
    
    if action == 1:
        reward = X[-1] - 4.0  # 4.0 is the spread of the trade
    
    if self._episode_ended: 
            return ts.termination(np.array([self._state], dtype=np.int32), reward)
    return ts.transition(
        np.array([self._state], dtype=np.int32), reward, discount=1.0)